<a href="https://colab.research.google.com/github/aneeshcheriank/pytorch_study/blob/main/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- torch.utils.data.DataLoader: an iterable around the Dataset
- torch.utils.data.DataSet: stores the samples and their currespoinding labels

In [47]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

- [torchvision datasets](https://pytorch.org/vision/stable/datasets.html)

In [48]:
# download the training data from open datasets
training_data = datasets.FashionMNIST(
    root='data' # the folder in which the data is stored
    , train=True # training data
    , download=True # if the data is not present it will download the data
    , transform = ToTensor(),
)

# Download test data from open datasets
testing_data = datasets.FashionMNIST(
    root='data' # the folder in which the data is stored
    , train=False # labeled data
    , download=True
    , transform = ToTensor(),
)

In [49]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(testing_data, batch_size=batch_size)

x, y = iter(train_dataloader).next()
print(f'shape of x: {x.shape}')
print(f'shape of x: {y.shape}')

shape of x: torch.Size([64, 1, 28, 28])
shape of x: torch.Size([64])


In [50]:
# get the training device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

# Define the model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [51]:
# optimizer and loss
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3) #pass model_parameters and learning rate

In [52]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train() # why?
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # compute the loss
    pred = model(X)
    loss = loss_fn(pred, y) # predicted, actual in tf it is the opposite

    # Backpropogation
    optimizer.zero_grad() # to clear the optimzier
    loss.backward()
    optimizer.step() # apply the gradients to the trainable weights

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f'loss: {loss} [{current}/{size}]')


- model.train()
  - dropouts and batchnormaliztion behaves differently during training and evaluation
  - so the model should know the training and evaluation
  - hence there is model.train() and model.eval()

In [53]:
def test(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f'Test Error:\n Accruacy: {100*correct}% Avg loss: {test_loss}\n')

In [54]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss, optimizer)
    test(test_dataloader, model, loss, optimizer)
print("Done!")

Epoch 1
-------------------------------
loss: 2.2985124588012695 [0/60000]
loss: 2.288897752761841 [6400/60000]
loss: 2.2618870735168457 [12800/60000]
loss: 2.2628700733184814 [19200/60000]
loss: 2.240830421447754 [25600/60000]
loss: 2.202150583267212 [32000/60000]
loss: 2.2237823009490967 [38400/60000]
loss: 2.1805951595306396 [44800/60000]
loss: 2.187969923019409 [51200/60000]
loss: 2.152620315551758 [57600/60000]
Test Error:
 Accruacy: 41.760000000000005% Avg loss: 2.1443387459797463

Epoch 2
-------------------------------
loss: 2.1573503017425537 [0/60000]
loss: 2.149087429046631 [6400/60000]
loss: 2.0751163959503174 [12800/60000]
loss: 2.102557897567749 [19200/60000]
loss: 2.041294574737549 [25600/60000]
loss: 1.978084683418274 [32000/60000]
loss: 2.012072801589966 [38400/60000]
loss: 1.9211896657943726 [44800/60000]
loss: 1.9384064674377441 [51200/60000]
loss: 1.8696942329406738 [57600/60000]
Test Error:
 Accruacy: 55.08% Avg loss: 1.8555260441105836

Epoch 3
-------------------

In [55]:
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [56]:
torch.save(model.state_dict(), 'model.pth')
print("saved the pytorch model to model.pth")

saved the pytorch model to model.pth


In [57]:
model = NeuralNetwork() # need to create a structure
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [58]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = testing_data[0][0], testing_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
